In [1]:
import os
os.chdir(os.path.abspath('../'))

In [2]:
import torch
import json
from torch_scatter import scatter_add
from qm9_allprop import QM9_allprop
from xgnn import xgnn_poly_bde

In [31]:
args = json.load(open(f'./ckpt/BDE_model_HS/args.json','rt'))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
attr_size = 338
model = xgnn_poly_bde(conv_layers=args['conv_layers'], sbf_dim=args['sbf_dim'], rbf_dim=args['rbf_dim'], in_channels=args['in_channels'], heads=args['heads'], embedding_size=args['embedding_size'], attr_size=attr_size, device=device).to(device)

In [32]:
ckpt = torch.load('./ckpt/BDE_model_HS/ckpt/ckpt_best.pth')
model.load_state_dict(ckpt['model'])
model.eval()

xgnn_poly_bde(
  (AF): SiLU()
  (emb_block): EmbeddingBlock(
    (AF): SiLU()
    (embedding): Embedding(10, 128, padding_idx=0, max_norm=3.0, scale_grad_by_freq=True)
    (lin): Linear(in_features=128, out_features=128, bias=True)
  )
  (envelop_function): poly_envelop()
  (sbf_layer): F_B_2D(
    (envelope): poly_envelop()
  )
  (rbf_layer): RadialBasis()
  (fin_model): SBFTransformer_bde(
    (edgenn): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): SiLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
    )
    (convs): ModuleList(
      (0): SBFTransformerConv(128, 8, heads=16)
      (1): SBFTransformerConv(128, 8, heads=16)
      (2): SBFTransformerConv(128, 8, heads=16)
      (3): SBFTransformerConv(128, 8, heads=16)
    )
    (readouts): ModuleList(
      (0): AtomWise(
        (mlp): ModuleList(
          (0): Linear(in_features=128, out_features=128, bias=True)
          (1): SiLU()
          (2): Linear(in_features=128,

In [34]:
def comparison_dataset(file_name,start_atom_id,end_atom_id,prop_len):
    dataset = QM9_allprop(input_file=f'./raw/{file_name}',prop_len=prop_len)
    dataset.data.atom_pos = dataset.data.atom_pos.float()
    dataset.data.edge_attr = dataset.data.edge_attr.float()

    with torch.no_grad():
        for i,data in enumerate(dataset):
            if model.mat_trans.weight.size()[1]==169:
                data['edge_attr'] = data['edge_attr'][:,:169]
                print(data['edge_attr'].size())
            data = data.to('cuda')
            atom_se, bond_se, bond_et = model(data)
            if i == 0:
                break
    edge_1 = torch.tensor([start_atom_id,end_atom_id])
    edge_2 = torch.tensor([end_atom_id,start_atom_id])
    edge_index = dataset.data.edge_index.T
    edge_1_id = -1
    edge_2_id = -1
    for i,edge in enumerate(edge_index):
        if (edge == edge_1).all():
            edge_1_id = i
        elif (edge == edge_2).all():
            edge_2_id = i
    bdepred = bond_et.squeeze()
    print(edge_1_id)
    print(edge_2_id)
    BDE = (-bdepred[edge_1_id] - bdepred[edge_2_id]).item()

    return BDE


In [52]:
final = comparison_dataset(file_name='bde_10.xyz',prop_len=1 ,start_atom_id=0,end_atom_id=1)
print(final/0.04336414)

0
12
100.79960142076152
